In [1]:
'''
This  replaces popnet as preprocessing script for SNPs from the UTR regions.
The main difference between the two is that we want to translate the stuff from
the cds, but not the UTR. So here we're just going to measure the number of non-ref
positions and use that as a metric. 
'''

import pandas as pd, numpy as np, re, sys
from pathlib import Path
from multiprocessing import Pool
from collections import Counter

In [2]:
in_path = Path('/d/data/plasmo/nat_out/v4')
in_file = in_path / 'utr_by_genes.tsv'
out_path = in_path / 'utr_scores.tsv'

In [3]:
#import file
in_df = pd.read_csv(in_file, sep = '\t', index_col = [0,1])

In [4]:
def toScore(row):
    #converts one row to a score according to some schele
    def getSingleScore(snp, ref):
        #here we define the scoring scheme
        #non-ref + 2, AT to CG, or reverse, +4, going in and out of C + 1
        score = 0
        if ref == snp:
            return 0
        else:
            return 1
#             if ref == 'C' or snp == 'C':
#                 score += 3
#             else:
#                 score += 2
                    
#         if ((ref == 'C' or ref == 'G') and (snp == 'A' or snp == 'T')):
#             return score + 4
#         elif ((ref == 'A' or ref == 'T') and (snp == 'C' or snp == 'C')):
#             return score + 4
#         else:
#             return score
            
    #the rest
    ref = Counter(row.values).most_common()[0][0] #ref is the most common element.. probably
    return row.apply(getSingleScore, 1, args = (ref,))

def worker(args):
    #hard coded args
    name = args[0]
    snps_df = args[1]
    
    scores = snps_df.apply(toScore, axis = 1)
    result = scores.apply(sum, axis = 0)
    
    result = pd.DataFrame(result, columns = [name]).T
    
    return result

by_genes = in_df.groupby('CHROM') #each iteration yields (name, df)
with Pool(6) as pool:
    res_dfs = pool.map(worker, by_genes)

In [5]:
res = pd.concat(res_dfs)
res.index = res.index.map(lambda x: x + '_UTR')
res.T.to_csv(out_path, sep='\t')